In [53]:
import requests

In [54]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)

# Save to file (optional)
with open("tinyshakespeare.txt", "w", encoding="utf-8") as f:
    f.write(response.text)


In [55]:
# Load into a variable
text = response.text

# Preview the first 500 characters
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [56]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

encoding = {}
decoding = {}

for i, char in enumerate(chars):
    encoding[char] = i

for i, char in enumerate(chars):
    encoding[i] = char



In [57]:
def encode(text):
    encoded = []
    for c in text:
        encoded.append(encoding[c])

    return encoded

def decode(text):
    decoded = []
    for c in text:
        decoded.append(encoding[c])

    return decoded

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
['h', 'i', 'i', ' ', 't', 'h', 'e', 'r', 'e']


In [58]:
import torch 
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [59]:
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [60]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [61]:
x = train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is ", context, " the target is ", target)

when input is  tensor([18])  the target is  tensor(47)
when input is  tensor([18, 47])  the target is  tensor(56)
when input is  tensor([18, 47, 56])  the target is  tensor(57)
when input is  tensor([18, 47, 56, 57])  the target is  tensor(58)
when input is  tensor([18, 47, 56, 57, 58])  the target is  tensor(1)
when input is  tensor([18, 47, 56, 57, 58,  1])  the target is  tensor(15)
when input is  tensor([18, 47, 56, 57, 58,  1, 15])  the target is  tensor(47)
when input is  tensor([18, 47, 56, 57, 58,  1, 15, 47])  the target is  tensor(58)


In [63]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else test_data
    ix  = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
xb, yb = get_batch("train")
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print("when input is ", context.tolist(), " the target is ", target)

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is  [24]  the target is  tensor(43)
when input is  [24, 43]  the target is  tensor(58)
when input is  [24, 43, 58]  the target is  tensor(5)
when input is  [24, 43, 58, 5]  the target is  tensor(57)
when input is  [24, 43, 58, 5, 57]  the target is  tensor(1)
when input is  [24, 43, 58, 5, 57, 1]  the target is  tensor(46)
when input is  [24, 43, 58, 5, 57, 1, 46]  the target is  tensor(43)
when input is  [24, 43, 58, 5, 57, 1, 46, 43]  the target is  tensor(39)
when input is  [44]  the target is  tensor(53)
when input is  [44, 53]  the target is  tensor(56)
when input is  [44, 53, 56]  